## シミュレーションを流すノートブック

In [ ]:
#path
using CUDA
using EzXML
using MDToolbox
using Statistics
device!(1)
pdb_filepath = "./alanine-dipeptide-nowater.pdb" #pdbのパス
top = readpdb(pdb_filepath)
ff_start_filepath  = "./data/amber14/protein.ff14SB.xml" #startのffのパス
out_dir = "sim_dihedral" #出力するディレクトリ
isdir(out_dir) || mkdir(out_dir)

#setting
njobs = 10 #並列して流すシミュレーションの数
nsteps = 2_500_000_000 #シミュレーションのステップ数
gpu_id = "n4"
update_ratio = 0.5 #forcefieldをどの程度変えるか

## シミュレーション

In [ ]:
# 一つのシミュレーションを流すジョブスクリプト
# ff_filepath　　　  :xml formatの力場ファイルのパス
# out_traj_dir      :アウトプットするディレクトリのパス
# i                 :流すシミュレーションの通し番号

function run_job(ff_filepath, traj_filepath, log_filepath, i)
    sbatch_file = "temp_$(i).sh"
    
    open(sbatch_file, "w") do of
        println(of, "#!/bin/bash")
        println(of, "#SBATCH -p all")
        println(of, "#SBATCH -J sim$(i) # job name")
        println(of, "#SBATCH -n 1  # num of total mpi processes")
        println(of, "#SBATCH -c 1  # num of threads per mpi processes")
        println(of, "#SBATCH -o $(log_filepath)")
        println(of, "export CUDA_VISIBLE_DEVICES=$(i)")
        #println(of, "python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath) $(nsteps)")
        println(of, "time python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath) $(nsteps)")
    end
    sleep(5)
    run(`sbatch --gpus-per-node=1 -w $(gpu_id) $(sbatch_file)`)
    #run(`sbatch -w $(gpu_id) $(sbatch_file)`)
    sleep(5)
    rm(sbatch_file)
end

## ffを作成する関数

In [ ]:
# 二面角phi,psiの値を変えてxmlファイルを作成する関数
# 値は元の値からupdate_ratioを参照して変える

phi_atom_type = ["protein-C", "protein-N", "protein-CX", "protein-C"]
psi_atom_type = ["protein-N", "protein-CX", "protein-C", "protein-N"]
function create_ff(ff_input_filepath, ff_output_filepath, update_ratio)
    input_xml = readxml(ff_input_filepath)
    output_xml = deepcopy(input_xml)
    xmlroot = root(output_xml)
    children = elements(xmlroot)
    children_name = nodename.(children)
    torsion_index = children_name .== "PeriodicTorsionForce"
    torsion = children[torsion_index][1] #これでtorsion内の要素にアクセスできる

    for ff_params in eachelement(torsion)
        params_name = [nodename(i) for i in eachattribute(ff_params)]
        params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        atom_type = [params_content[params_name .== "type$(i)"][1] for i in 1:4]
        if atom_type == phi_atom_type
            for a in eachattribute(ff_params)
                if occursin(r"k.*", a.name)
                    #ここにkを変える処理を入力
                    k = parse(Float32, a.content)
                    k_updated = k * (1 + update_ratio * (2 * rand() - 1))
                    a.content = string(k_updated)
                end
            end
            params_name = [nodename(i) for i in eachattribute(ff_params)]
            params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        end
        #ここから処理を書き忘れていたため追加した部分
        if atom_type == psi_atom_type
            for a in eachattribute(ff_params)
                if occursin(r"k.*", a.name)
                    #ここにkを変える処理を入力
                    k = parse(Float32, a.content)
                    k_updated = k * (1 + update_ratio * (2 * rand() - 1))
                    a.content = string(k_updated)
                end
            end
            params_name = [nodename(i) for i in eachattribute(ff_params)]
            params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        end
        #ここまで
    end
    write(ff_output_filepath, output_xml)
end

In [ ]:
#=
phi_atom_type = ["protein-C", "protein-N", "protein-CX", "protein-C"]
psi_atom_type = ["protein-N", "protein-CX", "protein-C", "protein-N"]

# こちらの関数はkの値を+-2.5の範囲で変える
function create_ff_2(ff_input_filepath, ff_output_filepath, update_ratio)
    input_xml = readxml(ff_input_filepath)
    output_xml = deepcopy(input_xml)
    xmlroot = root(output_xml)
    children = elements(xmlroot)
    children_name = nodename.(children)
    torsion_index = children_name .== "PeriodicTorsionForce"
    torsion = children[torsion_index][1] #これでtorsion内の要素にアクセスできる

    for ff_params in eachelement(torsion)
        params_name = [nodename(i) for i in eachattribute(ff_params)]
        params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        atom_type = [params_content[params_name .== "type$(i)"][1] for i in 1:4]
        if atom_type == phi_atom_type
            for a in eachattribute(ff_params)
                if occursin(r"k.*", a.name)
                    #ここにkを変える処理を入力
                    k = parse(Float32, a.content)
                    k_updated = k + rand(-2.5:0.001:2.5)
                    if k_updated < 0.0
                        k_updated = 0.0
                    end
                    a.content = string(k_updated)
                end
            end
            params_name = [nodename(i) for i in eachattribute(ff_params)]
            params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        end
        #ここから処理を書き忘れていたため追加した部分
        if atom_type == psi_atom_type
            for a in eachattribute(ff_params)
                if occursin(r"k.*", a.name)
                    #ここにkを変える処理を入力
                    k = parse(Float32, a.content)
                    k_updated = k + rand(-2.5:0.001:2.5)
                    if k_updated < 0.0
                        k_updated = 0.0
                    end
                    a.content = string(k_updated)
                end
            end
            params_name = [nodename(i) for i in eachattribute(ff_params)]
            params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        end
        #ここまで
    end
    write(ff_output_filepath, output_xml)
end
=#

In [ ]:
#二面角ϕ,ψに関連するパラメータはn,θ_zero,kの３つ
#最適化を目指すのはk

abstract type AbstractParam end

struct ParamCPU{T<:AbstractFloat}<:AbstractParam
    n::Vector{T}
    theta_zero::Vector{T}
    k::Vector{T}
end

struct Param{T<:AbstractFloat}<:AbstractParam
    n::CuArray{T}
    theta_zero::CuArray{T}
    k::CuArray{T}
end

function init_Param(ff_array::AbstractVector, gpu::Bool=false, T::DataType=Float64)
    n = T[]
    theta_zero = T[]
    k = T[]
    
    for line in ff_array
        #println(line)      
        if(occursin(r"type.*", line[1]))
            continue
        elseif(occursin(r"periodicity.*", line[1]))
            push!(n, parse(T, line[2]))
        elseif(occursin(r"phase.*", line[1]))
            push!(theta_zero, parse(T, line[2]))
        elseif(occursin(r"k.*", line[1]))
            push!(k, parse(T, line[2]))
        else
            println("ERROR")
        end
    end
    
    if(gpu) 
        n = adapt(CuArray, n)
        theta_zero = adapt(CuArray, theta_zero)
        k = adapt(CuArray, k)
        ff_param = Param(n, theta_zero, k)
        return ff_param
    end
    
    ff_param = ParamCPU(n, theta_zero, k)
    return ff_param
end

#力場ファイルから対象のパラメータを返す関数
function input_ff(ff_filepath; gpu=false::Bool)
    xml = readxml(ff_filepath)
    xmlroot = root(xml)
    
    children = elements(xmlroot)
    children_name = nodename.(children)
    torsion_index = children_name .== "PeriodicTorsionForce"
    torsion = children[torsion_index][1]
    
    phi_atom_type = ["protein-C", "protein-N", "protein-CX", "protein-C"]
    psi_atom_type = ["protein-N", "protein-CX", "protein-C", "protein-N"]
    
    ff_phi = []
    ff_psi = []
    for ff_params in eachelement(torsion)
        params_name = [nodename(i) for i in eachattribute(ff_params)]
        params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        atom_type = [params_content[params_name .== "type$(i)"][1] for i in 1:4]
        #println(atom_type)
        if(atom_type == phi_atom_type)
            ff_phi = [(params_name[i], params_content[i]) for i in 1:length(params_name)]
        end
        if(atom_type == psi_atom_type)
            ff_psi = [(params_name[i], params_content[i]) for i in 1:length(params_name)]
        end
    end
    return init_Param(ff_phi, gpu), init_Param(ff_psi, gpu)
end

In [ ]:
for i in 1:njobs
    sim_dir = joinpath(out_dir, "sim_$(i)")
    isdir(sim_dir) || mkdir(sim_dir)

    ff_filepath = joinpath(sim_dir, "sim_$(i).xml")
    create_ff(ff_start_filepath, ff_filepath, update_ratio)

    traj_filepath = joinpath(sim_dir, "traj_$(i)")
    log_filepath = joinpath(sim_dir, "sim_$(i).log")

    run_job(ff_filepath, traj_filepath, log_filepath, i)
end